## Baixa dados sobre Covid da web

In [2]:
import requests

def download_file(id, destination):
    URL = "https://drive.google.com/uc?id=1oiBDNZD65P3OCmOc1xrOlhL391MDcm-o&export=download"

    session = requests.Session()
    response = session.get(URL, params={'id': id})
    token = get_confirm_token(response)
    if(token):
        params = {'id': id, 'confirm': token}
        response = session.get(URL, params=params)
    save_response_content(response, destination)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)


if __name__ == "__main__":
    file_id = '1oiBDNZD65P3OCmOc1xrOlhL391MDcm-o'
    destination = 'dados_covid.csv'
    download_file(file_id, destination)


## Cria banco de dados

In [72]:
import sqlite3
from sqlite3 import Error
import os

#https://www.sqlitetutorial.net/sqlite-python/


def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


def getPath():
    return os.path.join(os.getcwd(), 'dw_prova_grupo.db')

if __name__ == '__main__':
    print(getPath())
    conn = create_connection(getPath())

c:\Users\arthu\Desktop\2021.1\Xexeo\dw_trabalho_final_grupo\dw_prova_grupo.db


In [73]:

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


def getPath():
    return os.path.join(os.getcwd(), 'dw_prova_grupo.db')

if __name__ == '__main__':
    print(getPath())
    conn = create_connection(getPath())

c:\Users\arthu\Desktop\2021.1\Xexeo\dw_trabalho_final_grupo\dw_prova_grupo.db


## Cria tabela estrela

In [74]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

In [75]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [76]:
def main():
    database = getPath()

    sql_create_vacinacao_covid_brasil = """ CREATE TABLE IF NOT EXISTS vacinacao_covid_brasil (
                                                fk_paciente_id text,
                                                fk_vacina_codigo integer,
                                                fk_estabelecimento_id integer,
                                                data_importacao_rnds text,
                                                sistema_origem text,
                                                id_sistema_origem integer,
                                                FOREIGN KEY (fk_paciente_id) REFERENCES paciente(paciente_id),
                                                FOREIGN KEY (fk_vacina_codigo) REFERENCES vacina(vacina_codigo),
                                                FOREIGN KEY (fk_estabelecimento_id) REFERENCES estabelecimento(estabelecimento_id)
                                            );"""

    sql_create_paciente_table = """ CREATE TABLE IF NOT EXISTS paciente (
                                        paciente_id text PRIMARY KEY,
                                        document_id text,
                                        paciente_idade integer,
                                        paciente_datanascimento text,
                                        paciente_enumsexobiologico char,
                                        paciente_racacor_valor text,
                                        paciente_racacor_codigo integer,
                                        paciente_endereco_coibgemunicipio integer,
                                        paciente_endereco_copais integer,
                                        paciente_endereco_nmmunicipio text,
                                        paciente_endereco_nmpais text,
                                        paciente_endereco_uf text,
                                        paciente_endereco_cep text,
                                        paciente_nacionalidade_enumnacionalidade char
                                    );"""

    sql_create_vacina_table = """ CREATE TABLE IF NOT EXISTS vacina (
                                    vacina_codigo integer PRIMARY KEY AUTOINCREMENT,
                                    vacina_nome text,
                                    vacina_numdose integer,
                                    vacina_descricao_dose text,
                                    vacina_dataaplicacao text,
                                    vacina_fabricante_referencia text,
                                    vacina_fabricante_nome text,
                                    vacina_lote text,
                                    vacina_categoria_nome text,
                                    vacina_categoria_codigo integer,
                                    vacina_grupoatendimento_nome text,
                                    vacina_grupoatendimento_codigo integer
                                );"""

    sql_create_estabelecimento_table = """ CREATE TABLE IF NOT EXISTS estabelecimento (
                                            estabelecimento_id integer PRIMARY KEY AUTOINCREMENT,
                                            estabelecimento_municipio_nome text,
                                            estabelecimento_municipio_codigo integer,
                                            estabelecimento_valor integer,
                                            estabelecimento_nofantasia text,
                                            estabelecimento_razaosocial text,
                                            estabelecimento_uf text
                                        );"""

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        # create historico table
        create_table(conn, sql_create_vacinacao_covid_brasil)
        create_table(conn, sql_create_paciente_table)
        create_table(conn, sql_create_vacina_table)
        create_table(conn, sql_create_estabelecimento_table)
    else:
        print("Error! cannot create the database connection.")

    #conn.commit()
    #conn.close()

In [77]:
if __name__ == '__main__':
    main()

## Insere dados na tabela

In [80]:
import pandas as pd

#function that insert csv data into database
def insert_data(df, table_name):
    conn = create_connection(getPath())
    df.to_sql(table_name, conn, if_exists='append', index=False)

paciente_atributos = ['document_id','paciente_idade','paciente_datanascimento','paciente_enumsexobiologico','paciente_racacor_valor','paciente_racacor_codigo','paciente_endereco_coibgemunicipio','paciente_endereco_copais','paciente_endereco_nmmunicipio','paciente_endereco_nmpais','paciente_endereco_uf','paciente_endereco_cep','paciente_nacionalidade_enumnacionalidade']

vacina_atributos = ['vacina_nome','vacina_numdose','vacina_descricao_dose','vacina_dataaplicacao','vacina_fabricante_referencia','vacina_fabricante_nome','vacina_lote','vacina_categoria_nome','vacina_categoria_codigo','vacina_grupoatendimento_nome','vacina_grupoatendimento_codigo'] 

estabelecimento_atributos = ['estabelecimento_municipio_nome', 'estabelecimento_municipio_codigo', 'estabelecimento_valor', 'estabelecimento_razaosocial', 'estabelecimento_uf']

dados_paciente = pd.read_csv('dados_covid.csv', usecols = paciente_atributos, sep = ';')
insert_data(dados_paciente, 'paciente')
dados_vacina = pd.read_csv('dados_covid.csv', usecols = vacina_atributos, sep = ';')
insert_data(dados_vacina, 'vacina')
dados_estabelecimento = pd.read_csv('dados_covid.csv', usecols = estabelecimento_atributos, sep = ';')
insert_data(dados_estabelecimento, 'estabelecimento')
#dados_covid.to_sql('paciente', conn, if_exists='append', index=False)
#insert_data(dados_covid, 'paciente', paciente_atributos)
